**Le machine learning représente l'outil de base du Data Scientist. **

Bien que le machine learning soit généralement utilisé comme une boite noire (on n'invente pas de nouveaux algorithmes tous les jours), il est utile de se familiariser avec le fonctionnement de différents algorithmes pour développer un savoir-faire sur la pertinence de tel ou tel algorithme pour un cas donné.

Nous allons voir dans ce notebook comment se construit un modèle de machine learning, en commençant par considérer le modèle le plus simple possible : une régression linéaire pour de l'apprentissage supervisé, afin de ne pas s'encombrer de détails mathématiques complexes. 


## Définition du modèle linéaire <a id='model'></a> 

$$y \sim f_{\theta}(x)$$

avec comme fonction f une forme linéaire

$$f_{\theta}(x) = \theta_0 + \theta_1 x +\epsilon $$


### Hypothèses du modèle linéaire statistique

* les erreurs suivent une loi normale de moyenne nulle
* la variance est la même pour tous (homoscédasticité) : la variance est la même pour l'ensemble des termes d'erreures gaussiens
$Var(\epsilon_i)=\sigma$
* les termes d'erreurs pour les différents $x_i$ sont indépendants les uns des autres

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def linear(x, params=(0,1), var=1):
    """Generate a linear function f(x)=a*x+b+N(0,1)
    
    Args:
        x (numpy.array()) : vector used to generate the output
        params (tuple of size 2) : b=params[0] and a=params[1]
        var (int) : noise spread 
    
    Returns:
        numpy.array()
    """
    a, b = params[1], params[0]
    return b + a*x + var*np.random.normal(size=len(x))

In [ ]:
x = 10*np.random.random(50)
x.sort()
my_params = (0,10)
y = linear(x, my_params, var = 10)

In [ ]:
plt.scatter(x,y)
plt.show()

## Méthode des moindres carrés

Afin de fixer les coefficients, nous définissons une mesure qui nous permet de connaître de façon empirique la qualité de l'ajustement.


Quelques constats :
1. Aucune droite ne peut passer par tous les points, vu qu'ils ne sont pas alignés
1. Chaque droite passe plus près de certains que d'autres
1. Nous décrétons pour le moment que la meilleure droite est celle qui passe le plus près de l'ensemble des points

Pour chaque x, nous allons calculer la distance cumulée entre $y_i$ projeté sur la droite à l'abscisse $x_i$, et minimiser cette quantité

$$Erreur(\theta) = \frac{1}{N}\sum_i (y_i-f_{\theta}(x_i))^2$$

* Les erreurs s'ajoutent toujours
* Pénalité sur les déviations les plus larges
* La fonction carré est dérivable

Nous souhaitons trouver les paramètres $\hat \theta$ tq $Erreur(\theta) \ge Erreur(\hat \theta)$

#### Commençons par définir quelques fonctions

In [ ]:
def linear_hypothesis(x, params=(0,1)):
    """Linear function f(x)=a*x+b
    
    Args:
        x (numpy.array()) : vector used to generate the output
        params (tuple of size 2) : b=params[0] and a=params[1]
    
    Returns:
        numpy.array()
    """
    a, b = params[1], params[0]
    return b + a*x

In [ ]:
def cost(x, y, func, func_args=None):
    """Cost function associated with (x,y) couple and function to fit
    
    Args:
        x : feature vector 
        y : explained variable
        func : target function
        func_args : arguments of function
        
    Returns:
        scalar value of cost
    """
    if func_args:
        return 1./len(x)*sum(np.square(y-func(x,func_args)))
    else :
        return 1./len(x)*sum(np.square(y-func(x)))

In [ ]:
cost(x,y,linear_hypothesis, (5,10))

In [ ]:
cost(x,y,linear_hypothesis, (0,1))

#### Recherche exhaustive

In [ ]:
def params_search():
    """2d scan for possible values and return best ones
    """
    errors = []
    params = []
    for p0 in np.linspace(0,10):
        for p1 in np.linspace(0,10):
            p = (p0,p1)
            c = cost(x, y, linear_hypothesis, p)
            errors.append(c)
            params.append(p)
    return params[errors.index(min(errors))]

best = params_search()
print("Paramètres optimisant les moindres carrés :", best)
plt.scatter(x,y)
plt.plot(x,best[0]+best[1]*x)
plt.show()

#### Recherche aléatoire de paramètres

In [ ]:
def random_params_search():
    errors = []
    params = []
    #choix arbitraire de recherche entre 0 et 10
    for p in 10*np.random.rand(100,2):
        p = tuple(p) 
        c = cost(x,y, linear, p)
        errors.append(c)
        params.append(p)
    return params[errors.index(min(errors))]

best = random_params_search()
print("Paramètres optimisant les moindres carrés :", best)
plt.scatter(x,y)
plt.plot(x,best[0]+best[1]*x)
plt.show()

On peut chercher longtemps, surtout si l'on n'a pas une bonne idée des valeurs des paramètres que l'on cherche.

Les problèmes de recherche d'optimum et de racines de fonction sont particulièrement courants. De nombreuses méthodes existent, qui dépassent le cadre de ce cours.

## Descente de gradient

On démarre avec des paramètres $\theta_i$ aléatoires.

Plutôt que de tester de nouvelles combinaisons au hasard, nous allons mettre à jour les valeurs que nous avons en ajoutant ou soustrayant une certaine quantité dans la direction qui minimise l'erreur.

$\theta_{i+1} = \theta_i + \delta$

Si `Erreur` croît, alors sa dérivée est positive, et si elle décroit, alors elle est négative. Nous voulons donc aller dans la direction des dérivées décroissantes.

$$\delta \propto -\frac{\partial E}{\partial \theta_i}$$
<img src="img/2000px-Gradient_descent.svg.png" width="500">

Le gradient vient de la forme vectorielle.


In [ ]:
def gradient_descent(x, y, start_params, alpha=0.001, iterations=1000):
    params = list(start_params)
    N=len(y)
    error_history = np.array([]).reshape(0,3)
    
    for i in range(iterations):
        params[0] = params[0] - (alpha/N)*sum(linear_hypothesis(x, params) - y)
        params[1] = params[1] - (alpha/N)*(linear_hypothesis(x, params) - y).dot(x)
        error_history = np.vstack([error_history,
                                   [cost(x,y, linear, tuple(params)), 
                                    params[0], params[1]]])
    return params, error_history    

In [ ]:
p, error_history = gradient_descent(x, y, (5,2), alpha=0.01, iterations=1000)
print(p)

Vérifions maintenant notre régression, l'évolution de l'erreur ainsi que des paramètres pendant la descente.

In [ ]:
plt.scatter(x,y)
plt.plot(x, p[0]+p[1]*x, 'g')

plt.figure()
plt.plot(np.arange(50), error_history[:50,0])
plt.title("Erreur")
plt.xlabel("itération")


plt.figure()
plt.plot(error_history[:,1], label='b')
plt.plot(error_history[:,2], label='a')

plt.xlabel("itération")
plt.ylabel("valeur")
plt.legend()

plt.show()

Pour aller plus loin avec l'utilisation par descente de gradient, qui joue un rôle centrale aujourd'hui dans l'optimisation des réseaux de neurones profonds : http://ruder.io/optimizing-gradient-descent/

## Régression avec sklearn et cross-validation

Nous pouvons faire la même chose, mais avec la librairie sklearn. Il faut cependant transformer le vecteur X en entrée, en passant d'une liste de nombres à une liste de feature vectors (liste de liste), même si il n'y a qu'un seul élément dans notre feature vector.

```
[[],
 [],
 [],
 ...
 []]
```

Il est en effet plutôt rare de n'avoir qu'un seul élément en entrée.

In [ ]:
print(x[:10])

In [ ]:
x_sk = x.reshape(-1,1)
print(x_sk[:10])

Afin d'illustrer l'intérêt de la cross-validation et le phénomène d'overfitting, on va utiliser un modèle plus complexe

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor()
forest.fit(x_sk, y)

In [ ]:
plt.scatter(x,y)
plt.plot(x, forest.predict(x_sk))

In [ ]:
cost(x,y, linear_hypothesis, (p[0], p[1]))

In [ ]:
cost(x_sk,y, forest.predict)

La random forest bat à plate couture le modèle linéaire. Mais nous allons maintenant tester le pouvoir prédictif, et la capacité à généraliser à des données qui n'ont pas été utilisées pour l'apprentissage. Pour cela nous allons

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)
x_train_sk = x_train.reshape(-1, 1)
x_test_sk = x_test.reshape(-1, 1)

In [ ]:
forest.fit(x_train_sk, y_train)

In [ ]:
plt.scatter(x_train, y_train)
plt.scatter(x_test, y_test)
plt.show()

In [ ]:
cost(x_train_sk, y_train, forest.predict)

In [ ]:
cost(x_test_sk, y_test, forest.predict)

In [ ]:
cost(x_test, y_test, linear_hypothesis, (p[0], p[1]))

In [ ]:
cost(x_train, y_train, linear_hypothesis, (p[0], p[1]))

L'erreur entre le train et le test est plus importante dans le cas de la random forest.